In [1]:
#imports
#from jupyter_plotly_dash import JupyterDash
import dash
import dash_table
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly
import pandas as pd

#import stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [2]:
df = pd.read_csv('zipcode-data.csv')

In [3]:
df.head()

,Zip Code,Area Name,Short Name,% Lacking Broadband Internet,Mean Income Past 12 Months
0,38126,South Forum - Washington Heights,South Forum,83.08,26346
1,38106,South Memphis,South Memphis,82.23,31403
2,38108,Hollywood - Hyde Park - Nutbush,Hyde Park,73.56,31278
3,38114,Rozelle - Castalia - Orange Mound - Fairhills ...,Orange Mound,70.64,38428
4,38105,Uptown - Pinch District,Pinch District,69.82,36980


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 5 columns):
Zip Code                        34 non-null int64
Area Name                       34 non-null object
Short Name                      34 non-null object
% Lacking Broadband Internet    34 non-null float64
Mean Income Past 12 Months      34 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.4+ KB


In [5]:
df['Zip Code'] = df['Zip Code'].astype(str)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 5 columns):
Zip Code                        34 non-null object
Area Name                       34 non-null object
Short Name                      34 non-null object
% Lacking Broadband Internet    34 non-null float64
Mean Income Past 12 Months      34 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.4+ KB


In [7]:
###### display function to render dashboard directly to notebook
from IPython import display
import os
def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = None):
    in_binder ='JUPYTERHUB_SERVICE_PREFIX' in os.environ if in_binder is None else in_binder
    if in_binder:
        base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
        url = 'https://hub.mybinder.org{}'.format(base_prefix)
        app.config.requests_pathname_prefix = base_prefix
    else:
        url = 'http://localhost:%d' % port
        
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [8]:
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

In [ ]:
colors = {
    'background': '#000000',
    'text': '#5d76a9',
    'label': '#f5b112'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.Div([
        html.Div([
            html.H1(children='Memphis Zip Code Data',
                style={
                    'textAlign': 'center',
                    'font-family':'Verdana',
                    'color': colors['text'],
                    'padding-top': 20
                }),
            html.P(children='Visualizing the Digital Divide in Memphis, TN', 
                style={
                    'textAlign': 'center',
                    'font-size': 24,
                    'font-family':'Verdana',
                    'color': colors['text'],
                    'padding-bottom': 10
            }), 
            html.P(['The data below was gathered from the U.S. Census Bureau table B28002, "PRESENCE AND TYPES OF INTERNET SUBSCRIPTIONS IN HOUSEHOLD".',html.Br(),'The data was filtered by zip code to view data for all 34 zip codes in the Memphis area.',html.Br(),'The complete derived datatable can be viewed below.'],
                style={
                    'margin-left': 100,
                    'margin-right': 100,
                    'font-size': 12,
                    'font-family':'Verdana',
                    'textAlign': 'center',
                    'color': colors['text']
            }),
            html.Label([html.A('Zip code data source', href='https://www.memphisrealtysearch.com/memphis-zip-code-map/', target="_blank")],
                style={
                    'font-size': 10,
                    'font-family':'Verdana',
                    'textAlign': 'center',
                    'color': colors['text']
            }),
            html.Label([html.A('Click here to view the data', href='https://data.census.gov/cedsci/table?q=b28002&tid=ACSDT1Y2018.B28002&vintage=2018&hidePreview=true&moe=false', target="_blank")],
                style={
                    'font-size': 10,
                    'font-family':'Verdana',
                    'textAlign': 'center',
                    'color': colors['text'],
                    'padding-bottom': 20
            })
        ], className='row'),
###############################################################      
        html.Div([
            dcc.Dropdown(
                id='demo-dropdown',
                options=[{'label': i, 'value': i} for i in df['Zip Code']],
                value=['38126', '38139'],
                multi=True
            ),
        html.Div(id='dd-output-container')]),
###############################################################        
        html.Div(children='', style={
                'padding': 15
            }),
###############################################################
        html.Div([
            html.Div([
                dcc.Graph(
                    id='graph1'
                )
            ], className='six columns'
            ),
###############################################################  
            html.Div([
                dcc.Graph(
                    id='graph2'
                )
            ], className='six columns'
            ),
###############################################################
            html.Div([
               dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    style_cell={'padding': '5px'},
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(212, 225, 250)'
                        }
                    ],
                    style_header={
                        'fontWeight': 'bold'
                    },
                    style_cell_conditional=[
                        {
                            'if': {'column_id': c},
                            'textAlign': 'left'
                        } for c in ['Zip Code', 'Area Name', 'Short Name']
                    ]),
            ], className='twelve columns'),
###############################################################
            html.Div([
                html.Footer(children='Created by Zach Cornelison', style={
                    'margin-top': 40,
                    'backgroundColor': 'black',
                    'color': 'white',
                    'font-size': 8,
                    'width': '100%',
                    'textAlign': 'center'
                }),
                html.Footer([html.A('Github Repository', 
                                    href='https://github.com/zachcornelison/Memphis-Zipcode-Data-Dashboardn', 
                                    target="_blank")], style={
                    'backgroundColor': 'black',
                    'color': 'white',
                    'font-size': 8,
                    'width': '100%',
                    'textAlign': 'center'
                })
            ], className='twelve columns'),
###############################################################  
        ], className='row')
    ], className='ten columns offset-by-one'),    
])

@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)
###############################################################  

@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_image_src(selector):
    filtered_data = df.loc[df['Zip Code'].isin(selector), 
                            ['Short Name', '% Lacking Broadband Internet', 'Zip Code']]
    figure = {
        'data': [{'x': [area_name], 
                  'y': [percent], 
                  'type': 'bar', 
                  'name': zip_code}
                  for area_name, percent, zip_code in filtered_data.to_numpy()
        ],
        'layout': {
            'title': 'Percent of Homes Lacking Broadband Internet',
            "titlefont": {
                "size": 20,
                'fontWeight': 'bold'
            },
            'yaxis' : dict(
                title='Percent Without Broadband',
                titlefont=dict(
                family='Verdana',
                size=16
            ))
        }
    }
    return figure
###############################################################  
@app.callback(
    dash.dependencies.Output('graph2', 'figure'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_image_src(selector):
    filtered_data = df.loc[df['Zip Code'].isin(selector), 
                            ['Short Name', 'Mean Income Past 12 Months', 'Zip Code']]
    figure = {
        'data': [{'x': [area_name], 
                  'y': [income], 
                  'type': 'bar', 
                  'name': zip_code}
                  for area_name, income, zip_code in filtered_data.to_numpy()
        ],
        'layout': {
            'title': 'Mean Income Last 12 Months',
            "titlefont": {
                "size": 20,
                'fontWeight': 'bold'
            },
            'yaxis' : dict(
                title='Income',
                titlefont=dict(
                family='Verdana',
                size=16
            ))
        }
    }
    return figure

In [ ]:
show_app(app)

Open in new window

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2020 14:34:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14:34:10] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 14: